The manager of your internship asks you to create programming code to calculate book value of a
bond. You need to create a function that calculates the book value of a bond immediately after the tth coupon (using the method and formula from STAT470). The function should incorporate
parameters F, C, n, r, i and t (notation consistent with STAT470). Test that your function works by
providing a test calculation by hand and comparing it with the value generated by the function.

In [223]:
# Shawdi Keshavarz
# a angle n-t --> (1-v^t)/i
def book_value(F, C, n, r, i, t):
    v = 1/(1+i)
    # Shawdi Keshavarz
    a_angle = (1-(v**(n-t)))/i
    return round((F*r*a_angle) + (C*((1+i)**-(n-t))),2)

# example from slides 10/20/23 Shawdi Keshavarz
print(book_value(1000,1000,20,.05, .06, 14))

# class activity 14 example
print(book_value(5000, 5000, 10, .0325, .07, 6))

950.83
4364.9


The amortization method we covered in STAT470 involves annual end-of-year loan/debt payments,
with the interest portion, principal portion, and outstanding principal (P) all calculated as of the end
of each year (at the time of each payment). This question involves amortization methods involving
annual beginning-of-year and annual mid-year loan/debt payments, requiring you to calculate the
interest portion, principal portion, and P at the time of each loan/debt payment.
Sydney works for a benefits consulting firm and is preparing for an upcoming client meeting to show
the client different options for amortizing debt. Sydney’s project manager asks her to create an
interactive program for the client to create amortization tables. The program needs to allow the
client to enter:
- The initial amount of debt
- The number of years over which the debt is being amortized
- An option to choose whether the interest rate is convertible monthly, quarterly or semiannually
    - After selecting the interest conversion, the client can enter a nominal rate
    
After the information is entered, the program needs to generate 3 separate amortization
schedules/tables with all table values calculated (amount of annual debt payment, interest portion,
principal portion, P). One table should assume annual beginning-of-year payments, one table should
assume mid-year (7/1/XX) payments, and one table should assume end-of-year payments. The
interest portion, principal portion, and P should all be calculated at the time of each payment. In all
three tables, the initial debt amount entered is at 1/1/ in the first year. The tables should be created
for a maximum of 100 rows (years). NOTE: Students can use MS Excel for this problem if they prefer.


In [224]:
# Shawdi Keshavarz
import pandas as pd
import math

def calc_a(num_years, i_effective):
    v = 1/(1+i_effective)
    return (1-(v**num_years))/i_effective

# Shawdi Keshavarz
def calc_a_due(num_years, i_effective):
    v = 1/(1+i_effective)
    d = i_effective/(1+i_effective)
    return (1-(v**num_years))/d

# Shawdi Keshavarz
def end_year_schedule(init_debt:float, payment:float, interest_effective:float):
    duration = 0
    df=pd.DataFrame({'Duration':[duration], 'Payment':[0], 'iP':[0], 'pP':[0], 'P':[init_debt]})
    while df.iloc[-1]['P'] > 0:
        duration += 1
        prev_debt = df.iloc[-1]['P']
        ip=round(prev_debt*interest_effective,2)
        pp=round(payment-ip,2)
        p=round(prev_debt-pp,2)
        if p < 0:
            p=0
        df.loc[len(df.index)] = [math.trunc(duration), payment, ip, pp, p]
    df['Duration']=df['Duration'].astype(int)
    return df

# Shawdi Keshavarz
def mid_year_schedule(init_debt:float, payment: float, interest_effective:float):
    df=pd.DataFrame({'Duration':[0], 'Payment':[0], 'iP':[0], 'pP':[0], 'P':[init_debt]})
    
    new_loan=init_debt*((1+interest_effective)**.5)
    ip = round((new_loan * interest_effective),2)
    pp = round(abs(payment-ip),2)
    p = round((new_loan-payment),2)
    df.loc[1] = [1, payment, ip, pp, p]
    
    p1 = df.iloc[1]['P']
    ip2 = round((p1*interest_effective),2)
    pp2 = round((payment-ip2),2)
    p2 = (p1*(1+interest_effective)) - payment 
    df.loc[2] = [2, payment, ip2, round(pp2,2), round(p2,2)]
    
    duration = 2
    print(df.iloc[-1]['P'])
    while df.iloc[-1]['P'] > 0:
        duration += 1
        prev_debt = df.iloc[-1]['P']
        ip=round(prev_debt*interest_effective,2)
        pp=round(payment-ip,2)
        p=round(prev_debt-pp,2)
        if p < 0:
            p=0
        df.loc[len(df.index)] = [math.trunc(duration), payment, ip, pp, p]
    df['Duration']=df['Duration'].astype(int)
    return df

# Shawdi Keshavarz
def beg_year_schedule(init_debt:float, payment:float, interest_effective:float):
    duration = 0
    df = pd.DataFrame({'Duration':[duration], 'Payment':[payment], 'iP':[0], 'pP':[payment], 'P':[init_debt-payment]})
    while df.iloc[-1]['P'] > 0:
        duration += 1
        prev_debt = df.iloc[-1]['P']
        ip=round(prev_debt*interest_effective,2)
        pp=round(payment-ip,2)
        p=round(prev_debt-pp,2)
        if p < 0:
            p=0
        df.loc[len(df.index)] = [math.trunc(duration), payment, ip, pp, p]
    df['Duration']=df['Duration'].astype(int)
    return df


# Shawdi Keshavarz
def amortization_schedule(debt:float, num_years:int, i_rate:str, i_nom:float):
    print("Debt: " + str(debt) + " Number of Years: " + str(num_years))
    i_effective_ann = i_nom
    if i_rate == "m":
        i_effective_ann = ((1+(i_nom/12))**12)-1
    elif i_rate == "q":
        i_effective_ann = ((1+(i_nom/4))**4)-1
    elif i_rate == "s":
        i_effective_ann = ((1+(i_nom/2))**2)-1
        
    # Shawdi Keshavarz
    print("Annual End of Year Payment")
    payment = debt/calc_a(num_years,i_effective_ann)
    print(end_year_schedule(debt, round(payment,2), i_effective_ann))
    
    print("\nAnnual Beginning of Year Payment")
    payment = debt/calc_a_due(num_years,i_effective_ann)
    print(beg_year_schedule(debt, round(payment,2), i_effective_ann))
    
    print("\nAnnual Mid-Year Payments")
    payment = (debt*((1+i_effective_ann)**.5))/calc_a_due(num_years, i_effective_ann)
    print(mid_year_schedule(debt, round(payment,2), i_effective_ann))
    
    
# Shawdi Keshavarz
amortization_schedule(5000,5,'a', .12)
print("\n")
amortization_schedule(5000,6, 'a', .14)
print("\n")
amortization_schedule(50000,10, 's',.13)
amortization_schedule(1000, 10, 'a', .1)



Debt: 5000 Number of Years: 5
Annual End of Year Payment
   Duration  Payment      iP       pP        P
0         0     0.00    0.00     0.00  5000.00
1         1  1387.05  600.00   787.05  4212.95
2         2  1387.05  505.55   881.50  3331.45
3         3  1387.05  399.77   987.28  2344.17
4         4  1387.05  281.30  1105.75  1238.42
5         5  1387.05  148.61  1238.44     0.00

Annual Beginning of Year Payment
   Duration  Payment      iP       pP        P
0         0  1238.44    0.00  1238.44  3761.56
1         1  1238.44  451.39   787.05  2974.51
2         2  1238.44  356.94   881.50  2093.01
3         3  1238.44  251.16   987.28  1105.73
4         4  1238.44  132.69  1105.75     0.00

Annual Mid-Year Payments
3147.92
   Duration  Payment      iP       pP        P
0         0     0.00    0.00     0.00  5000.00
1         1  1310.64  634.98   675.66  3980.86
2         2  1310.64  477.70   832.94  3147.92
3         3  1310.64  377.75   932.89  2215.03
4         4  1310.64  265.80 

This question involves determining the break-even interest rate in a sinking fund so that the
borrower is “making money” by using a sinking fund rather than a traditional loan payment process.

Create a function/program that compares the total annual outlay for a borrower using a sinking
fund compared to a borrower using a traditional loan payment process. The function/program
should allow the following information to be entered:
- The initial amount of the loan
- The number of years over which the loan is being repaid
- An option to choose whether the loan interest rate is convertible monthly, quarterly,
semiannually, or annually
    - After selecting the interest conversion, enter a nominal rate

You can assume that the annual loan payment, the annual interest payment under the sinking fund
method and the annual sinking fund deposit are always made at the end of each year. The
function/program should generate the following output:

A. The sinking fund interest rates resulting in a “break-even” total annual outlay compared to
the traditional loan payment method (i.e., total annual outlay = annual loan payment)
- Regardless of the loan interest conversion selected, 4 sinking fund interest rates should
be provided: convertible monthly, quarterly, semiannually, or annually

B. The annual loan payment, annual interest payment under the sinking fund method and
annual sinking fund deposit for the “break-even” situation (in this output, the total annual
outlay should = annual loan payment)

C. In addition to the “break-even” situation, the same output should be generated (rates in
part A and payment amounts in part B) for the situation when the borrower saves $500,
$1,000 and $2,000 annually. The output for the $500 savings should only be created if the
annual loan payment is > $750. Similarly, the output for the $1,000 and $2,000 savings
should only be created if the annual loan payment is > $1,500/$3,000.
NOTE: Students can use MS Excel for this problem if they prefer.


In [225]:
# Shawdi Keshavarz
import math
import numpy as np
import pandas as pd
from sympy import *

dc = {'m':12, 'q':4, 's':2, 'a':1}

def convert_effective(i_rate:str, i_des:str, i_nom:float):
    i_rate=dc[i_rate]
    i_des=dc[i_des]
    left_side =(1+(i_nom/i_rate))**i_rate
    solved = (left_side**i_des)-1
    return round(solved,4)

def convert_nominal(i_rate:str, i_des:str, i_eff:float):
    i_rate = dc[i_rate]
    i_des=dc[i_des]
    left_side = (1+i_eff)**(i_rate/i_des)
    solved = left_side-1
    return round(solved,4)

def calc_a(num_years, i_effective):
    v = 1/(1+i_effective)
    return (1-(v**num_years))/i_effective


    
# Shawdi Keshavarz
def sink(init_debt:float, num_years:int, i_rate:str, i_nom_l:float):
    i_effective_monthly = convert_effective(i_rate, 'm', i_nom_l)
    i_effective_quart = convert_effective(i_rate, 'q', i_nom_l)
    i_effective_semi = convert_effective(i_rate, 's', i_nom_l)
    i_effective_ann = convert_effective(i_rate, 'a', i_nom_l)
    
    #annual loan payment
    ann_loan_pay = round((init_debt/calc_a(num_years, i_effective_ann)),2)
    interest_paymentAnn = init_debt*i_effective_ann
    
    print("Part A:")
    if i_rate =="a":
        #find sinking fund interest convertible annually
        interest_paymentAnn=init_debt*i_effective_ann
        right_sideAnn = ann_loan_pay-interest_paymentAnn
        i = Symbol('i')
        eq = ((((1+i)**num_years)-1)/i) - (init_debt/right_sideAnn)
        break_even = solve(eq,i)
        # Shawdi Keshavarz
        sol = []
        for ans in break_even:
            try:
                answer = float(ans)
                if answer < 1 and answer > 0:
                    sol.append(round(answer,4))
            except:
                continue
        
        print("annual sinking fund interest rate: ", sol[0])
        print("semi-annual sinking fund interest: ", convert_nominal('a','s',sol[0]))
        print("quarterly sinking fund interest: ", convert_nominal('a','q',sol[0]))
        print("month sinking fund interest: ", convert_nominal('a','m',sol[0]))
    elif i_rate == "s":
        #find sinking fund interest convertible semi-annually
        interest_paymentSemi=init_debt*i_effective_semi
        right_sideSemi = ann_loan_pay-interest_paymentSemi
        i = Symbol('i')
        eq = ((((1+i)**num_years*2)-1)/i) - (init_debt/right_sideSemi)
        break_even = solve(eq,i)
        # Shawdi Keshavarz
        sol = []
        for ans in break_even:
            try:
                answer = float(ans)
                if answer < 1 and answer > 0:
                    sol.append(round(answer,4))
            except:
                continue
        
        print("annual sinking fund interest rate: ", convert_nominal('s','a',sol[0]))
        print("semi-annual sinking fund interest: ", sol[0])
        print("quarterly sinking fund interest: ", convert_nominal('s','q',sol[0]))
        print("month sinking fund interest: ", convert_nominal('s','m',sol[0]))
    elif i_rate == "q":
        #find sinking fund interest convertible quarterly
        interest_paymentQuart=init_debt*i_effective_quart
        right_sideQuart = ann_loan_pay-interest_paymentQuart
        i = Symbol('i')
        eq = ((((1+i)**num_years*4)-1)/i) - (init_debt/right_sideQuart)
        break_even = solve(eq,i)
        
        sol = []
        for ans in break_even:
            try:
                answer = float(ans)
                if answer < 1 and answer > 0:
                    sol.append(round(answer,4))
            except:
                continue
        
        print("annual sinking fund interest rate: ", convert_nominal('q','a',sol[0]))
        print("semi-annual sinking fund interest: ", convert_nominal('q','s',sol[0]))
        print("quarterly sinking fund interest: ", sol[0])
        print("month sinking fund interest: ", convert_nominal('q','m',sol[0]))
    elif i_rate == "m":
        #find sinking fund interest convertible monthly
        interest_paymentMon=init_debt*i_effective_monthly
        right_sideMon = ann_loan_pay-interest_paymentMon
        i = Symbol('i')
        eq = ((((1+i)**num_years*12)-1)/i) - (init_debt/right_sideMon)
        break_even = solve(eq,i)
        
        sol = []
        for ans in break_even:
            try:
                answer = float(ans)
                if answer < 1 and answer > 0:
                    sol.append(round(answer,4))
            except:
                continue
        
        print("annual sinking fund interest rate: ", convert_nominal('m','a',sol[0]))
        print("semi-annual sinking fund interest: ", convert_nominal('m','s',sol[0]))
        print("quarterly sinking fund interest: ", convert_nominal('m','q',sol[0]))
        print("month sinking fund interest: ", sol[0])

    print("\nPart B: ")
    print("Annual Loan Payment: ", ann_loan_pay)
    print("Annual Sinking Fund Interest Payment: ", interest_paymentAnn)
    print("Annual Sinking Fund Deposit: ", ann_loan_pay-interest_paymentAnn)
    
    print("\nPart C:")
    i = Symbol("i")
    ann_loan_pay=0
    interest_payment = 0
    right_side=0
    if i_rate == 'a':
        ann_loan_pay = init_debt/calc_a(num_years, i_effective_ann)
        interest_payment = init_debt*i_effective_ann
    elif i_rate == 's':
        num_years = num_years*2
        ann_loan_pay = init_debt/calc_a(num_years, i_effective_semi)
        interest_payment = init_debt*i_effective_semi
        # Shawdi Keshavarz
    elif i_rate == 'q':
        num_years = num_years*4
        ann_loan_pay = init_debt/calc_a(num_years, i_effective_quart)
        interest_payment = init_debt*i_effective_quart
    elif i_rate == 'm':
        num_years = num_years*12
        ann_loan_pay = init_debt/calc_a(num_years, i_effective_monthly)
        interest_payment = init_debt*i_effective_monthly
        
    if (ann_loan_pay >= 3000):
        right_side = ann_loan_pay-interest_payment-2000
    elif (ann_loan_pay <3000 and ann_loan_pay >= 1500):
        right_side = ann_loan_pay-interest_payment-1000
    elif (ann_loan_pay < 1500 and ann_loan_pay >= 500):
        right_side = ann_loan_pay-interest_payment-750
    
    eq = ((((1+i)**num_years)-1)/i) - (init_debt/right_side)
    break_uneven = solve(eq,i)
    sol2 = []
    for ans in break_uneven:
        try:
            answer = float(ans)
            if answer < 1 and answer > 0:
                sol2.append(round(answer,4))
        except:
            continue
    if i_rate == 'a':
        print("annual sinking fund interest rate: ", sol2[0])
        print("semi-annual sinking fund interest: ", convert_nominal('a','s',sol2[0]))
        print("quarterly sinking fund interest: ", convert_nominal('a','q',sol2[0]))
        print("month sinking fund interest: ", convert_nominal('a','m',sol2[0]))
        # Shawdi Keshavarz
    elif i_rate == 's':
        print("annual sinking fund interest rate: ", convert_nominal('s','a',sol2[0]))
        print("semi-annual sinking fund interest: ", sol2[0])
        print("quarterly sinking fund interest: ", convert_nominal('s','q',sol2[0]))
        print("month sinking fund interest: ", convert_nominal('s','m',sol2[0]))
    elif i_rate == 'q':
        print("annual sinking fund interest rate: ", convert_nominal('q','a',sol2[0]))
        print("semi-annual sinking fund interest: ", convert_nominal('q','s',sol2[0]))
        print("quarterly sinking fund interest: ", sol2[0])
        print("month sinking fund interest: ", convert_nominal('q','m',sol2[0]))
    elif i_rate == 'm':
        print("annual sinking fund interest rate: ", convert_nominal('m','a',sol2[0]))
        print("semi-annual sinking fund interest: ", convert_nominal('m','s',sol2[0]))
        print("quarterly sinking fund interest: ", convert_nominal('m','q',sol2[0]))
        print("month sinking fund interest: ", sol2[0])
        # Shawdi Keshavarz
    print("Annual Loan Payment: ", round(ann_loan_pay,2))
    print("Annual Sinking Fund Interest Payment: ", round(interest_paymentAnn,2))
    print("Annual Sinking Fund Deposit: ", round((right_side),2))
    
    
    
    
    
    
    
    
    
    
    
# sanity check 
# print(convert_effective('s','a', .13))
# x = Symbol('x')
# print( solve( (x*calc_a(15, .17)-15000) , x) )


sink(30000, 5, 'a', .05)



Part A:
annual sinking fund interest rate:  0.05
semi-annual sinking fund interest:  0.0247
quarterly sinking fund interest:  0.0123
month sinking fund interest:  0.0041

Part B: 
Annual Loan Payment:  6929.24
Annual Sinking Fund Interest Payment:  1500.0
Annual Sinking Fund Deposit:  5429.24

Part C:
annual sinking fund interest rate:  0.2829
semi-annual sinking fund interest:  0.1327
quarterly sinking fund interest:  0.0643
month sinking fund interest:  0.021
Annual Loan Payment:  6929.24
Annual Sinking Fund Interest Payment:  1500.0
Annual Sinking Fund Deposit:  3429.24
